In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("D:/CARRERA/TESIS/forest_explainer_tesis/datasets/Titanic/DataSet_Titanic.csv")

In [3]:
from sklearn.model_selection import train_test_split

data = df.drop(columns="Sobreviviente")
x_train, x_test, y_train, y_test = train_test_split(data,
                                                    df["Sobreviviente"], test_size=0.2,
                                                    random_state=123)

In [4]:
import joblib
from sklearn.ensemble import RandomForestClassifier

random_forest_model: RandomForestClassifier = joblib.load(
    "D:/CARRERA/TESIS/forest_explainer_tesis/datasets/Titanic/titanic.joblib")

In [5]:
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc

In [6]:
options = [
    {
        'label': "Label 1"
    },
    {
        'label': "Label 2"
    },
]

y_pred = random_forest_model.predict_proba(x_test)

In [7]:
options = [
    {
        'label': "Label 1"
    },
    {
        'label': "Label 2"
    },
]

y_pred = random_forest_model.predict_proba(x_test)

In [73]:
def create_curve(y_scores, y_true, options, pointers):
        # One hot encode the labels in order to plot them
        # y_onehot = pd.get_dummies(y_true)
        # print(y_onehot)

        data = []
        trace1 = go.Scatter(x=[0, 1], y=[0, 1],
                            mode='lines',
                            line=dict(dash='dash'),
                            showlegend=False

                            )

        data.append(trace1)
        cont = 0
        for i in range(y_scores.shape[1]):
            y_score = y_scores[:, i]
            
            pointer = pointers[i]

            fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=i)
            auc_score = auc(fpr, tpr)
            
            if pointer >= 0:
                name = f"{options[cont]['label']} (AUC={auc_score:.2f})"
                trace2 = go.Scatter(x=fpr, y=tpr,
                                    name=name,
                                    mode='lines')
                data.append(trace2)
                scatterPointer = int(len(fpr) * pointer / 100)
                print(scatterPointer)
                trace3 = go.Scatter(x=[fpr[scatterPointer]], y=[tpr[scatterPointer]],
                                    name=f"Sc{fpr[scatterPointer]}",)
                trace4 = go.Scatter(x=[0, fpr[scatterPointer]], y=[tpr[scatterPointer], tpr[scatterPointer]],
                                mode='lines',
                                name=f"Line{fpr[scatterPointer]}",
                                line=dict(dash='dash'),

                                )
                trace5 = go.Scatter(x=[fpr[scatterPointer], fpr[scatterPointer]], y=[0, tpr[scatterPointer]],
                                mode='lines',
                                name=f"Line{fpr[scatterPointer]}",
                                line=dict(dash='dash'),

                                )
                data.append(trace3)
                data.append(trace4)
                data.append(trace5)
            cont += 1

        layout = go.Layout(
            title='ROC-AUC curva',
            yaxis=dict(title='Tasa de Positivos'),
            xaxis=dict(title='Tasa de Falsos Positivos')
        )

        fig = go.Figure(data=data, layout=layout)

        return fig

In [74]:
fig = create_curve(y_scores=y_pred, y_true=y_test, options=options, pointers=[45,-1])
fig


17
